<a href="https://colab.research.google.com/github/tonycico0912/ChatGPT_Python/blob/main/%EA%B5%AD%ED%9A%8C%EB%B0%9C%EC%9D%98%EB%B2%95%EC%95%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from google.colab import files
from bs4 import BeautifulSoup
import openpyxl
from openpyxl.utils import get_column_letter

# API 호출에 필요한 기본 정보
API_KEY = '20d952279d7240239bdaf1d81cfc5985'  # API 키를 입력
AGE = '22'  # 22대 국회
BASE_URL = 'https://open.assembly.go.kr/portal/openapi/nzmimeepazxkubdpn'

# 수집할 의안 번호의 개수 입력
num_bills = int(input("수집할 의안 번호의 개수를 입력하세요: "))
latest_bill_no = 2203462  # 최신 의안 번호를 시작점으로 설정

# 수집할 의안 번호 리스트 생성
bill_numbers = [str(latest_bill_no - i) for i in range(num_bills)]

# 법안 데이터 저장 리스트
bill_data = []

# 각 의안 번호에 대해 정보 수집
for bill_no in bill_numbers:
    # API 호출 URL 생성
    url = f"{BASE_URL}?KEY={API_KEY}&Type=xml&AGE={AGE}&BILL_NO={bill_no}"

    # API 호출
    response = requests.get(url)
    response_text = response.text

    # XML 파싱
    root = ET.fromstring(response_text)

    # XML 파싱 후 필요한 데이터 추출
    for row in root.findall('row'):
        bill_name = row.find('BILL_NAME').text
        proposer = row.find('PROPOSER').text
        member_list_url = row.find('MEMBER_LIST').text
        committee = row.find('COMMITTEE').text if row.find('COMMITTEE') is not None else ""

        # 발의 의원 명단 URL을 통해 추가 정보 불러오기
        member_list_response = requests.get(member_list_url)
        member_list_html = member_list_response.text

        # HTML 파싱을 통해 발의 의원 명단 추출
        soup = BeautifulSoup(member_list_html, 'html.parser')
        member_list = soup.find_all('a')  # 의원 이름을 포함하는 링크 추출

        # 의원 명단 추출 후 괄호 전 정보만 가져오고, 최대 30명까지 리스트에 추가
        members = [member.text.strip().split('(')[0].strip() for member in member_list[:30]]

        # API 데이터 파싱하여 리스트에 저장
        bill_info = {
            'BILL_NO': bill_no,
            'BILL_NAME': bill_name,
            'PROPOSER': proposer,
            'COMMITTEE': committee
        }
        for i, member in enumerate(members):
            bill_info[f'MEMBER_LIST{i + 1}'] = member

        bill_data.append(bill_info)

# DataFrame으로 변환
df = pd.DataFrame(bill_data)

# 사용자로부터 파일 이름 입력 받기
file_name = input("저장할 엑셀 파일의 이름을 입력하세요 (확장자 제외): ") + ".xlsx"

# 엑셀 파일로 저장
df.to_excel(file_name, index=False)

# 엑셀 파일을 열어서 각 열의 너비를 자동으로 조정
wb = openpyxl.load_workbook(file_name)
ws = wb.active

for col in ws.columns:
    max_length = 0
    column = col[0].column_letter  # 엑셀의 열 번호를 가져옴
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column].width = adjusted_width

# 파일을 저장하고 다운로드 링크 생성
wb.save(file_name)
files.download(file_name)


수집할 의안 번호의 개수를 입력하세요: 100
저장할 엑셀 파일의 이름을 입력하세요 (확장자 제외): 최근발의법안_100


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import itertools
from google.colab import files

# 1. 사용자로부터 엑셀 파일 업로드
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# 2. 엑셀 파일을 불러옵니다.
df = pd.read_excel(file_name)

# 3. NetMiner용 데이터를 저장할 리스트 초기화
netminer_data = []

# 4. 각 행에 대해 데이터를 처리합니다.
for index, row in df.iterrows():
    # MEMBER_LIST1을 대표 발의자로 설정
    proposer = row['MEMBER_LIST1'].replace('의원', '').strip()

    # 나머지 MEMBER_LIST를 공동 발의자로 설정
    members = [row[f'MEMBER_LIST{i+1}'].replace('의원', '').strip() for i in range(1, len(row)-4) if pd.notna(row[f'MEMBER_LIST{i+1}'])]

    # 대표-공동발의 관계 추가
    for member in members:
        if member != proposer:
            netminer_data.append([proposer, member, "대표-공동발의", "Yes"])

    # 공동발의자 간의 관계 추가
    pairs = list(itertools.combinations(members, 2))
    for pair in pairs:
        netminer_data.append([pair[0], pair[1], "공동발의", "No"])

# 5. DataFrame으로 변환
netminer_df = pd.DataFrame(netminer_data, columns=["국회의원 1", "국회의원 2", "관계", "대표발의자 여부"])

# 6. 사용자로부터 저장할 파일 이름 입력 받기
output_file_name = input("저장할 파일 이름을 입력하세요 (확장자 제외): ") + ".txt"

# 7. 텍스트 파일로 저장
netminer_df.to_csv(output_file_name, sep='\t', index=False, encoding='utf-8')

# 결과 확인
print(netminer_df.head())

# 8. 파일 다운로드
files.download(output_file_name)
